# CrysFML: Neutrons vs X-rays

In [ ]:
# Misc
import sys, os
import numpy as np

# CrysFML
import CFML_api

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

In [ ]:
output_notebook()
FIGURE_WIDTH = 700
FIGURE_HEIGHT = 300

In [ ]:
def diffraction_pattern_xy(space_group, cell, atom_list, job_info):
    reflection_list = CFML_api.ReflectionList(cell, space_group, 
                                              True, 
                                              job_info)
    reflection_list.compute_structure_factors(space_group, 
                                              atom_list, 
                                              job_info)
    diffraction_pattern = CFML_api.DiffractionPattern(job_info,
                                                  reflection_list, 
                                                  cell.reciprocal_cell_vol)
    return diffraction_pattern.x, diffraction_pattern.ycalc

In [ ]:
def set_default_parameters(job_info):
    job_info.range_2theta = (10.0, 155.45)
    job_info.theta_step = 0.05
    job_info.lambdas = (1.9122, 1.9122)
    job_info.lambda_ratio = 1.0
    job_info.u_resolution = 0.1421
    job_info.v_resolution = -0.4181
    job_info.w_resolution = 0.3864
    job_info.x_resolution = 0.0
    job_info.y_resolution = 0.0904

## Define model

### Create space group object

In [ ]:
space_group = CFML_api.SpaceGroup('P n m a')
#space_group.print_description()

### Create unit cell object

In [ ]:
lengths = np.asarray([8.477,5.396,6.957], dtype='float32')
angles = np.asarray([90,90,90], dtype='float32')
unit_cell = CFML_api.Cell(lengths, angles)
#unit_cell.print_description()

### Create list of atoms

In [ ]:
cif_str = '''
loop_
 _atom_site_label
 _atom_site_type_symbol
 _atom_site_occupancy
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_adp_type
 _atom_site_U_iso_or_equiv
  Pb  Pb  1.0  0.1880  0.25    0.1668  Uiso  0.0169
  S   S   1.0  0.0650  0.25    0.6850  Uiso  0.0025
  O1  O   1.0  0.9080  0.25    0.5950  Uiso  0.0246
  O2  O   1.0  0.1935  0.25    0.5432  Uiso  0.0180
  O3  O   1.0  0.0814  0.0274  0.8085  Uiso  0.0165
'''
atom_list = CFML_api.AtomList(cif_str.split('\n'))
atom_list.set_mult_occ_cif(space_group)
#atom_list.print_description()

### Create job definition object

In [ ]:
cfl_str = '''
Title SrTiO3
'''
job_info = CFML_api.JobInfo(cfl_str.split('\n'))
job_info.print_description()

### Modify job definition object

In [ ]:
job_info.pattern_type = "NEUT_2THE"
set_default_parameters(job_info)
job_info.print_description()

## Calculate neutron powder diffraction pattern

### Plot calculated pattern

In [ ]:
x_data, y_data_neutrons = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons, legend_label='Neutrons', color='orangered', line_width=2)
show(fig)

## Calculate X-rays powder diffraction pattern

### Modify job definition object

In [ ]:
job_info.pattern_type = "XRAY_2THE"
job_info.print_description()

### Plot calculated pattern

In [ ]:
x_data, y_data_xrays = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_xrays, legend_label='Xrays / 500', color='olivedrab', line_width=2)
show(fig)

## Compare neutron and X-ray diffraction patterns

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons, legend_label='Neutrons', color='orangered', line_width=2)
fig.line(x_data, y_data_xrays/500, legend_label='Xrays / 500', color='olivedrab', line_width=2)
show(fig)

## Calculate neutron powder diffraction pattern (2nd iteration)

### Modify job definition object

In [ ]:
job_info.pattern_type = "NEUT_2THE"
job_info.print_description()

### Plot calculated pattern

In [ ]:
x_data, y_data_neutrons2 = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons2, legend_label='Neutrons II', color='orange', line_width=2)
show(fig)

## Calculate X-rays powder diffraction pattern (2nd iteration)

### Modify job definition object

In [ ]:
job_info.pattern_type = "XRAY_2THE"
job_info.print_description()

### Plot calculated pattern

In [ ]:
x_data, y_data_xrays2 = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_xrays2, legend_label='Xrays II / 500', color='darkgreen', line_width=2)
show(fig)

## Compare neutron and X-ray diffraction patterns (2nd iteration)

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons2, legend_label='Neutrons II', color='orange', line_width=2)
fig.line(x_data, y_data_xrays2/500, legend_label='Xrays II / 500', color='darkgreen', line_width=2)
show(fig)

## Compare neutron and X-ray diffraction patterns (all iterations)

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons, legend_label='Neutrons', color='orangered', line_width=2)
fig.line(x_data, y_data_xrays/500, legend_label='Xrays / 500', color='olivedrab', line_width=2)
fig.line(x_data, y_data_neutrons2, legend_label='Neutrons II', color='orange', line_width=2)
fig.line(x_data, y_data_xrays2/500, legend_label='Xrays II / 500', color='darkgreen', line_width=2)
show(fig)